# Geospatial Analysis with H3 Aggregation

This notebook demonstrates geospatial analysis capabilities of the Ocean Data Platform, including spatial filtering and H3 hexagonal grid aggregation.

**What you'll learn:**
- Query data using geospatial filters (within, intersects, contains)
- Aggregate data using H3 hexagonal grids
- Visualize spatial distributions on interactive maps

**Why H3?**

[H3](https://h3geo.org/) is Uber's hexagonal hierarchical spatial index. Benefits:
- Uniform cell shapes (unlike lat/lon grids)
- Multiple resolution levels (0-15)
- Efficient for spatial aggregation and analysis

**Prerequisites:**
- Running in ODP Workspace (auto-authenticated)
- Completed `01_catalog_discovery.ipynb`

## 1. Setup

In [ ]:
from odp.client import Client
from IPython.display import display
import pandas as pd
import numpy as np
import subprocess
import sys

# Initialize ODP client
client = Client()

# Auto-install visualization packages if missing
def ensure_package(package_name, import_name=None):
    """Install package if not available."""
    import_name = import_name or package_name
    try:
        __import__(import_name)
        return True
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package_name])
        return True

# Install and import visualization packages
try:
    ensure_package("folium")
    ensure_package("h3")
    import folium
    from folium.plugins import HeatMap
    import h3
    HAS_FOLIUM = True
    print("Folium and h3 ready for mapping")
except Exception as e:
    HAS_FOLIUM = False
    print(f"Visualization packages unavailable: {e}")

try:
    ensure_package("matplotlib")
    import matplotlib.pyplot as plt
    HAS_MPL = True
except:
    HAS_MPL = False

print("Client initialized")

## 2. Connect to Dataset

We'll use the PGS Biota dataset which contains marine mammal and turtle observations with geographic coordinates.

In [ ]:
# PGS Biota dataset - TABULAR access
# Note: This UUID doesn't appear in STAC catalog but works via SDK
# See debug cell below for investigation of TABULAR vs FILE UUIDs
DATASET_ID = "1d801817-742b-4867-82cf-5597673524eb"

dataset = client.dataset(DATASET_ID)
schema = dataset.table.schema()

if schema:
    print(f"Connected to TABULAR dataset")
    print(f"Columns: {[f.name for f in schema]}")
    stats = dataset.table.stats()
    if stats:
        print(f"Rows: {stats.num_rows:,}")
else:
    print("Dataset not accessible as tabular. Run debug cell below to investigate.")

In [ ]:
# DEBUG: Test if old and new UUIDs represent different access patterns
# Hypothesis: Same data may have separate TABULAR and FILE UUIDs

old_uuid = "1d801817-742b-4867-82cf-5597673524eb"  # Original PGS Biota
new_uuid = "b960c80e-7ead-47af-b6c8-e92a9b5ac659"  # PGS Brazil - Biota and Physics

print("Testing UUID access patterns...\n")

for label, uuid in [("Old UUID", old_uuid), ("New UUID", new_uuid)]:
    try:
        ds = client.dataset(uuid)
        schema = ds.table.schema()
        if schema:
            stats = ds.table.stats()
            rows = stats.num_rows if stats else "?"
            print(f"{label}: TABULAR ({rows} rows)")
            print(f"  Columns: {[f.name for f in schema][:5]}...")
            # If tabular, use this one
            if label == "Old UUID":
                DATASET_ID = uuid
                dataset = ds
                print(f"  → Using this dataset")
        else:
            files = ds.files.list()
            print(f"{label}: FILE ({len(files)} files)")
    except Exception as e:
        print(f"{label}: Error - {str(e)[:50]}")
    print()

## 3. Geospatial Filtering

ODP supports three spatial operators:
- **within**: Points/geometries fully inside a polygon
- **intersects**: Any overlap with a polygon
- **contains**: Polygon contains another geometry

In [ ]:
# Define a region of interest - Brazilian coast (where PGS data is located)
brazil_coast = "POLYGON((-48 -28, -32 -28, -32 0, -48 0, -48 -28))"

# Query data within the region
# Note: Adjust geometry column name based on your dataset schema
GEOMETRY_COL = "footprintWKT"  # For PGS Biota dataset

try:
    df = dataset.table.select(
        f"{GEOMETRY_COL} within $area",
        vars={"area": brazil_coast}
    ).all(max_rows=10000).dataframe()
    
    print(f"Found {len(df)} observations in region")
    print(f"\nColumns: {list(df.columns)}")
    df.head()
except Exception as e:
    print(f"Query error: {e}")
    print("\nTrying without spatial filter...")
    df = dataset.table.select().all(max_rows=1000).dataframe()
    print(f"Loaded {len(df)} rows")

In [ ]:
# Explore the data
if 'scientificName' in df.columns:
    print("Species observed:")
    print(df['scientificName'].value_counts().head(10))

if 'decimalLatitude' in df.columns and 'decimalLongitude' in df.columns:
    print(f"\nLatitude range: {df['decimalLatitude'].min():.2f} to {df['decimalLatitude'].max():.2f}")
    print(f"Longitude range: {df['decimalLongitude'].min():.2f} to {df['decimalLongitude'].max():.2f}")

## 4. H3 Hexagonal Aggregation

H3 aggregation groups data into hexagonal cells at a specified resolution (0-15).

| Resolution | Avg Hex Area | Use Case |
|------------|--------------|----------|
| 0 | 4,357,449 km² | Continental |
| 3 | 12,393 km² | Regional |
| 5 | 253 km² | Local |
| 7 | 5.16 km² | Detailed |
| 9 | 0.1 km² | Fine-grained |

In [ ]:
# Aggregate observations by H3 hexagon
# Resolution 5 gives ~250 km² hexagons - good for regional patterns
H3_RESOLUTION = 5

# Handle different h3 library versions (v3 vs v4 API)
def get_h3_cell(lat, lon, res):
    try:
        return h3.latlng_to_cell(lat, lon, res)  # v4+ API
    except AttributeError:
        return h3.geo_to_h3(lat, lon, res)  # v3 API

# NOTE: Server-side H3 aggregation is not yet compatible with visualization.
# The ODP API's h3() function returns internal indices, not H3 cell ID strings.
# Using client-side aggregation until server-side returns proper H3 hex strings.
# See: proposals/server_side_h3_aggregation.md for discussion.

if HAS_FOLIUM and 'decimalLatitude' in df.columns:
    print("Computing H3 cells client-side...")
    print(f"(Server-side H3 returns indices, not hex IDs - see proposals/server_side_h3_aggregation.md)")
    
    df['h3_cell'] = df.apply(
        lambda row: get_h3_cell(row['decimalLatitude'], row['decimalLongitude'], H3_RESOLUTION)
        if pd.notna(row['decimalLatitude']) and pd.notna(row['decimalLongitude']) else None,
        axis=1
    )
    
    h3_agg = df.groupby('h3_cell').size().reset_index(name='count')
    h3_agg = h3_agg[h3_agg['h3_cell'].notna()]
    
    print(f"H3 aggregation: {len(h3_agg)} hexagonal cells")
    print(f"\nTop 10 cells by observation count:")
    print(h3_agg.nlargest(10, 'count'))
else:
    h3_agg = None
    print("H3 aggregation requires h3 library and coordinate columns")

In [ ]:
# DEBUG: Investigate server-side H3 aggregation response
# Uncomment to see what the ODP API returns for h3() group_by

# try:
#     h3_server = dataset.table.aggregate(
#         group_by=f"h3({GEOMETRY_COL}, {H3_RESOLUTION})",
#         filter=f"{GEOMETRY_COL} IS NOT NULL",
#         aggr={"occurrenceID": "count"}
#     )
#     print("Server-side H3 aggregation result:")
#     print(f"Shape: {h3_server.shape}")
#     print(f"Columns: {list(h3_server.columns)}")
#     print(f"Dtypes:\n{h3_server.dtypes}")
#     print(f"\nFirst 10 rows:\n{h3_server.head(10)}")
#     print(f"\nSample values from first column: {h3_server.iloc[:5, 0].tolist()}")
# except Exception as e:
#     print(f"Server-side H3 error: {e}")

## 5. Visualize on Map

In [ ]:
if HAS_FOLIUM and 'decimalLatitude' in df.columns:
    # Calculate map center
    center_lat = df['decimalLatitude'].mean()
    center_lon = df['decimalLongitude'].mean()
    
    # Create base map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=5)
    
    # Add observation points
    for _, row in df.head(500).iterrows():  # Limit for performance
        if pd.notna(row['decimalLatitude']) and pd.notna(row['decimalLongitude']):
            folium.CircleMarker(
                location=[row['decimalLatitude'], row['decimalLongitude']],
                radius=3,
                color='blue',
                fill=True,
                popup=row.get('scientificName', 'Unknown')
            ).add_to(m)
    
    print(f"Map centered at ({center_lat:.2f}, {center_lon:.2f})")
    print(f"Showing {min(500, len(df))} observation points")
    display(m)
else:
    print("Folium not available or no coordinate columns found.")
    print("Install folium: pip install folium")

In [ ]:
# Visualize H3 hexagons
if HAS_FOLIUM and h3_agg is not None:
    # Handle different h3 library versions
    def get_h3_boundary(hex_id):
        try:
            return h3.cell_to_boundary(hex_id)  # v4+ API
        except AttributeError:
            return h3.h3_to_geo_boundary(hex_id, geo_json=False)  # v3 API
    
    m_hex = folium.Map(location=[center_lat, center_lon], zoom_start=5)
    
    # Plot top 100 hexagons by count
    for _, row in h3_agg.nlargest(100, 'count').iterrows():
        hex_id = row['h3_cell']
        count = row['count']
        
        boundary = get_h3_boundary(hex_id)
        coords = [[lat, lng] for lat, lng in boundary]
        
        # Color based on count
        color = 'red' if count > 10 else 'orange' if count > 5 else 'yellow'
        
        folium.Polygon(
            locations=coords,
            color=color,
            fill=True,
            fill_opacity=0.4,
            popup=f"Count: {count}"
        ).add_to(m_hex)
    
    print("H3 hexagon map (color = observation density)")
    display(m_hex)
else:
    print("H3 visualization requires successful aggregation in previous cell")

## 6. Aggregation with Statistics

Beyond counting, aggregate other statistics per hexagon.

In [ ]:
# Aggregate multiple statistics
# Example: If dataset has depth data
if 'minimumDepthInMeters' in df.columns:
    try:
        depth_agg = dataset.table.aggregate(
            group_by=f"h3({GEOMETRY_COL}, {H3_RESOLUTION})",
            filter=f"{GEOMETRY_COL} IS NOT NULL AND minimumDepthInMeters IS NOT NULL",
            aggr={
                "minimumDepthInMeters": "mean",
                "occurrenceID": "count"
            }
        )
        print("Depth statistics by H3 cell:")
        print(f"Columns returned: {list(depth_agg.columns)}")
        # Sort by count column (name may vary)
        count_col = [c for c in depth_agg.columns if 'count' in c.lower() or c == 'occurrenceID']
        if count_col:
            print(depth_agg.sort_values(count_col[0], ascending=False).head(10))
        else:
            print(depth_agg.head(10))
    except Exception as e:
        print(f"Aggregation error: {e}")
else:
    print("No depth column in this dataset")
    print(f"Available columns: {list(df.columns)}")

In [ ]:
# Aggregate by species within region
if 'scientificName' in df.columns:
    try:
        species_agg = dataset.table.aggregate(
            group_by="scientificName",
            filter=f"{GEOMETRY_COL} IS NOT NULL AND scientificName IS NOT NULL",
            aggr={
                "occurrenceID": "count"
            }
        )
        print("Observations by species:")
        print(f"Columns returned: {list(species_agg.columns)}")
        # Sort by count column (name may vary)
        count_col = [c for c in species_agg.columns if 'count' in c.lower() or c == 'occurrenceID']
        if count_col:
            print(species_agg.sort_values(count_col[0], ascending=False).head(15))
        else:
            print(species_agg.head(15))
    except Exception as e:
        print(f"Aggregation error: {e}")

## 7. Multi-Resolution Analysis

Compare patterns at different H3 resolutions.

In [ ]:
# Compare resolutions
if HAS_FOLIUM and 'decimalLatitude' in df.columns:
    # Handle different h3 library versions
    def get_h3_cell(lat, lon, res):
        try:
            return h3.latlng_to_cell(lat, lon, res)  # v4+ API
        except AttributeError:
            return h3.geo_to_h3(lat, lon, res)  # v3 API
    
    resolutions = [3, 5, 7]
    
    for res in resolutions:
        df[f'h3_res{res}'] = df.apply(
            lambda row, r=res: get_h3_cell(
                row['decimalLatitude'], 
                row['decimalLongitude'], 
                r
            ) if pd.notna(row['decimalLatitude']) else None,
            axis=1
        )
        unique_cells = df[f'h3_res{res}'].nunique()
        print(f"Resolution {res}: {unique_cells} unique cells")
else:
    print("Requires coordinate columns and h3 library")

## Summary

This notebook demonstrated:

1. **Geospatial Filtering**: Query data using `within`, `intersects`, `contains` operators with WKT polygons
2. **H3 Aggregation**: Group data into hexagonal cells at configurable resolutions
3. **Map Visualization**: Display observations and hex patterns on interactive maps
4. **Statistical Aggregation**: Calculate metrics (count, mean, etc.) per spatial cell
5. **Multi-Resolution**: Compare patterns at different spatial scales

## Next Steps

- **03_data_pipeline.ipynb**: Ingest files and transform into tabular data
- **04_multi_dataset_join.ipynb**: Combine multiple datasets for analysis

## Resources

- [H3 Documentation](https://h3geo.org/docs/)
- [ODP Python SDK - Aggregation](https://docs.hubocean.earth/python_sdk/intro/)
- [Folium Documentation](https://python-visualization.github.io/folium/)